<a href="https://colab.research.google.com/github/Anna172/ML/blob/master/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tqdm


**Данные**

Будем работать с датасетом от онлайн-кинотеатра Okko REKKO CHALLENGE.

Имеющиеся данные содержат несколько файлов. Рассмотрим всё, что у нас есть.

catalogue.json содержит анонимизированную метаинформацию о доступных в сервисе фильмах и сериалах:

In [2]:
catalogue = pd.read_json('data/catalogue.json').T

ValueError: ignored

In [ ]:
catalogue['element_uid'] = catalogue.index

In [ ]:
catalogue.head()



*   attributes — мешок атрибутов;
*   availability — доступность (может содержать значения purchase, rent и subscription);
*   duration — длительность фильма (для сериалов и многосерийных фильмов — длительность серии) в минутах, округлённая до десятков;
*   feature_1..5 — пять анонимизированных вещественных и порядковых признаков;
*   type — принимает значения movie, multipart_movie или series.

transactions.csv — список всех транзакций за определённый период времени:

In [ ]:
transactions = pd.read_csv('data/transactions.csv',
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'consumption_mode': 'category',
        'ts': np.float64,
        'watched_time': np.uint64,
        'device_type': np.uint8,
        'device_manufacturer': np.uint8
    }
)

In [ ]:
transactions.head()

In [ ]:
transactions.shape


*   element_uid — идентификатор элемента (фильма);
*   user_uid — идентификатор пользователя;
*   consumption_mode — тип потребления (P — покупка, R — аренда, S — просмотр по подписке);
*   ts — время совершения транзакции или начала просмотра в случае просмотра по подписке;
*   watched_time — число просмотренных по транзакции секунд;
*   device_type — анонимизированный тип устройства, с которого была совершена транзакция или начат просмотр;
*   device_manufacturer — анонимизированный производитель устройства, с которого была совершена транзакция или начат просмотр.

ratings.csv содержит информацию о поставленных пользователями оценках:

In [ ]:
ratings = pd.read_csv('data/ratings.csv')

In [ ]:
ratings.head()

In [ ]:
ratings.shape

*   rating — поставленный пользователем рейтинг (от 0 до 10).

Далее будем обучать модель на долю времени просмотра фильма. Для этого к матрице transactions добавим информацию о длительности фильма.

In [ ]:
transactions = transactions.merge(catalogue, how='left', on='element_uid')


В колонке transactions['duration'] могут быть нули, так как значения округляются до ближайшей десятки. Поэтому добавим 10 ко всем значениям. И умножим все значения transactions['duration'] на 60, чтобы перевести в секунды.

In [ ]:
transactions.loc[:, 'duration'] += 10
transactions['duration']  = transactions['duration']*60



Для простоты вычислений далее будем использовать не всех пользователей.

In [ ]:
transactions_small = transactions.loc[transactions.user_uid < transactions.user_uid.quantile(.01)]

In [ ]:
transactions.element_uid.nunique()

Чтобы оценивать качество предсказаний построенных алгоритмов, разобьем данные на обучающую и тестовую выборки. Для этого вычислим 75-й перцентиль признака ts и разделим данные transactions_small во времени, так, чтобы в обучающую выборку попало 75% данных, а в тестовую — остальные 25%.

In [ ]:
transactions_test = transactions_small.loc[transactions_small.ts > transactions_small.ts.quantile(.75)].reset_index()
transactions_train = transactions_small.loc[transactions_small.ts < transactions_small.ts.quantile(.75)].reset_index()

In [ ]:
transactions_train.shape

In [ ]:
transactions_train.user_uid.nunique()

In [ ]:
transactions_test.user_uid.nunique()

Данные ratings тоже необходимо сократить и исключить оценки из тестовой части:

In [ ]:
test_user_ids = set(transactions_test[['element_uid', 'user_uid']].itertuples(index=False))
train_user_ids = set(transactions_train[['element_uid', 'user_uid']].itertuples(index=False))
drop_index = []
for row in tqdm.tqdm(ratings.itertuples()):
    if (row.element_uid, row.user_uid) in test_user_ids or not (row.element_uid, row.user_uid) in train_user_ids:
        drop_index.append(row.Index)

In [ ]:
ratings.drop(drop_index, inplace=True)
ratings.reset_index(inplace=True)

In [ ]:
ratings.shape

На выделенной обучающей выборке необходимо обучить рекомендательную систему и предсказать top-20 наиболее релевантных для пользователя идентификаторов контента.

Получим значение целевой переменной для итогового алгоритма — долю времени просмотра фильма:

In [ ]:
transactions_train['target'] = transactions_train.watched_time/transactions_train.duration

In [ ]:
transactions_test['target'] = transactions_test.watched_time/transactions_test.duration

В поле target присутствуют значения больше 1, так как для сериалов и многосерийных фильмов используется средняя длина серии — заменим в таких случаях целевую переменную на 1:

In [ ]:
transactions_train.loc[transactions_train['target'] > 1, 'target'] = 1
transactions_test.loc[transactions_test['target'] > 1, 'target'] = 1

Далее в задании везде будем обозначать как $r_{ui}$ значение этой целевой переменной для пользователя $u$ и элемента $i$.

Задание 0 [0.5 балла]
Для оценки качества предсказаний будем использовать метрику $MNAP@20$, которую использовали в соревновании. Метрика отличается от оригинальной $MAP$ тем, что значение для каждого пользователя нормализуется не на константу, а на число потреблённых фильмов. Таким образом, вес угадывания одного фильма больше у пользователей, имеющих меньшее число просмотров.

$$MNAP@20 = \dfrac{1}{|U|}\sum_{u\in U}\dfrac{1}{\min(n_u, 20)}\sum_{i=1}^{20}r_u(i)p_u@i $$$$p_u@k = \dfrac{1}{k}\sum_{j=1}^kr_u(j)$$
$r_{u}(i)$ — потребил ли пользователь u контент, предсказанный ему на месте i (1 либо 0);
$n_u$ — количество элементов, которые пользователь потребил за тестовый период;
$U$ — множество тестовых пользователей.
Реализуйте функцию mnap_k, вычисляющую метрику качества $MNAP@K$, где

predictions — предсказания фильмов для пользователей List[List[int]];

target — фильмы которые посмотрел пользователь List[Set[int]].

In [ ]:
def mnap_k(predictions, target, k=20):
    #╰( ͡° ͜ʖ ͡° )つ──☆*:・
    return


Проведем элементарные проверки на корректность реализации:

In [ ]:
test = [list(np.arange(1, 21)), list(np.arange(2, 22)), list(np.arange(3, 23))]
target = [set(np.arange(1, 21)), set(np.arange(2, 22)), set(np.arange(3, 23))]
assert mnap_k(test, target) == 1.0

In [ ]:
test = [list(np.arange(1, 21)), list(np.arange(2, 22)), list(np.arange(3, 23))]
target = [set(np.arange(1, 11)), set(np.arange(2, 12)), set(np.arange(3, 13))]
assert mnap_k(test, target) == 1.0

In [ ]:
test = [list(np.arange(1, 21)), list(np.arange(2, 22)), list(np.arange(3, 23))]
target = [set(np.arange(2, 21)), set(np.arange(2, 22)), set(np.arange(3, 23))]
assert round(mnap_k(test, target), 3) == 0.954

Memory-based

Два пользователя похожи, если им нравятся одинаковые фильмы. Рассмотрим двух пользователей $u$ и $v$ и обозначим через $I_{uv}$ множество фильмов $i$, которые просмотрели пользователи

Реализуйте класс MemoryBased с методами:

__init__ — конструктор класса, принимает на вход матрицу целевых переменных $r_{ui}$;
user_similarity — возвращает вектор $w_u$ сходства пользователя $u$ и всех пользователей из обучающей выборки; если пользователь $u$ не встречался в обучающей выборке, то возвращайте нулевой вектор;
item_similarity — возвращает вектор $w_i$ сходства фильма $i$ и всех фильмов из обучающей выборки; если фильм $i$ не встречался в обучающей выборке, то возвращайте нулевой вектор.
Уделите особое внимание оптимизации кода. Используйте векторизацию и матричные вычисления, где это возможно.

In [ ]:
class MemoryBased():
    def __init__(self, ratings):
        self.ratings = ratings
        pass
        
    def user_similarity(self, test_user):
      """test_user - вектор соответствующий пользователю"""
        #╰( ͡° ͜ʖ ͡° )つ──☆*:・
        pass
        
    def item_similarity(self, test_item):
      """test_item - вектор соответствующий фильму"""
        #╰( ͡° ͜ʖ ͡° )つ──☆*:・
        pass

Проведем элементарные проверки на корректность реализации:

если считаем, что

$\bar r_u$ и $\bar r_v$ — средняя доля времени просмотра фильма для пользователей, посчитанная по всем просмотренным фильмам

$\bar r_i$ и $\bar r_j$ — средняя доля времени просмотра для фильмов, посчитанная по всем пользователям, которые смотрели фильм

In [ ]:
I = np.array([[0.5, 0.4, 0, 0.1], 
         [0, 0.1, 0.2, 0.5], 
         [0.5, 0.5, 0.4, 0],
         [0.5, 0.4, 0.5, 0.1]])
user_based = MemoryBased(I)
result = user_based.user_similarity(np.array([0.5, 0.4, 0, 0.1])
assert np.all(np.round(result, 2) == np.array([1.0, -0.94, 0.92, 0.97]))

если считаем, что

$\bar r_u$ и $\bar r_v$ — средняя доля времени просмотра фильма для пользователей, посчитанная по множеству $I_{uv}$

$\bar r_i$ и $\bar r_j$ — средняя доля времени просмотра для фильмов, посчитанная по множеству $I_{ij}$

In [ ]:
I = np.array([[0.5, 0.4, 0, 0.1], 
         [0, 0.1, 0.2, 0.5], 
         [0.5, 0.5, 0.4, 0],
         [0.5, 0.4, 0.5, 0.1]])
user_based = MemoryBased(I)
result = user_based.user_similarity(np.array([0.5, 0.4, 0, 0.1])
assert np.all(np.round(result, 2) == np.array([1.0, -1.0, 0.0, 1.0]))

User-Based

В подходе на основе сходств пользователей определяется множество $U(u_0)$ пользователей, похожих на данного

товаров с наибольшими значениями $p_i$. Данный подход позволяет строить рекомендации, если для данного пользователя найдутся похожие. Если же пользователь является нетипичным, то подобрать что-либо не получится.

Реализуйте класс UserBased, наследованный от MemoryBased. Считайте, что $\alpha = 0$.

Класс UserBased должен иметь метод реализованный recomendation_k с параметром k, возвращающий матрицу предсказаний размера N x k (для каждого пользователя из user_vectors предсказывать top-k фильмов).

In [ ]:
class UserBased(MemoryBased):
    def recomendation_k(self, user_vectors, k=20):
      """user_vectors - матрица размера N x M, где M - общее число фильмов, N - число пользователей, для которых делаем рекомендации"""
        #╰( ͡° ͜ʖ ͡° )つ──☆*:・
        pass

Проведем элементарные проверки на корректность реализации:

In [ ]:
I = np.array([[0.5, 0.4, 0, 0.1], 
         [0, 0.1, 0.2, 0.5], 
         [0.5, 0.5, 0.4, 0],
         [0.5, 0.4, 0.5, 0.1]])
item_based = UserBased(I)
result = item_based.recomendation_k(np.array([0.5, 0.4, 0, 0.1]), k=1)
assert np.all(result == np.array([2]))

Item-Based

Определяется множество фильмов, похожих на те, которые интересовали данного пользователя

Пользователю рекомендуются $$$фильмов с наибольшими значениями $p_i$.

Реализуйте класс ItemBased, наследованный от MemoryBased.

Класс ItemBased должен иметь метод реализованный recomendation_k с параметром k, возвращающий матрицу предсказаний размера N x k (для каждого пользователя из user_vectors предсказывать top-k фильмов).

In [ ]:
class ItemBased(MemoryBased):
    def recomendation_k(self, user_vectors, k=20):
      """user_vectors - матрица размера N x M, где M - общее число фильмов, N - число пользователей, для которых делаем рекомендации"""
        #╰( ͡° ͜ʖ ͡° )つ──☆*:・
        pass

Latent factor model (LFM)
В этом подходе значение целовой переменной $r_{ui}$ пользователя $u$, поставленная фильму $i$, ищется как скалярное произведение векторов $p_u$ и $q_i$ в некотором пространстве $R^K$ латентных признаков:$$
    r_{ui}
    \approx
    \langle p_u, q_i \rangle.
$$

Иными словами, модель находит пространство признаков, в котором мы описываем и фильмы, и пользователей и в котором целевая переменная является мерой близости между фильмами и пользователями.

Для настройки модели будем минимизировать следующий функционал:

$$
\label{eq:lfmReg}
    \sum_{(u, i) \in R}
        \left(
            r_{ui}
            - \langle p_u, q_i \rangle
        \right)^2
    +
    \lambda
    \sum_{u \in U}
        \|p_u\|^2
    +
    \mu
    \sum_{i \in I}
        \|q_i\|^2
    \to
    \min_{P, Q}
$$
В статье описан метод оптимизации ALS (Alternating Least Squares) для данного функционала. В методе проводятся $N$ итераций, в рамках каждой итерации сначала оптимизируется $p$ при фиксированном $q$, затем $q$ при фиксированном $p$.

Составим матрицу $P$ из векторов $p_u$ и матрицу $Q$ из векторов $q_i$. Матрицей $Q[u] \in R^{n_u×K}$ будем обозначать подматрицу матрицы $Q$ только для товаров, оцененных пользователем $u$, где $n_u$ – количество оценок пользователя $u$. Шаг перенастройки $p_u$ при фиксированной матрице $Q$ сводится к настройке Ridge-регрессии и выглядит так:$$A_u = Q[u]^T Q[u] $$$$d_u = Q[u]^Tr_u $$$$p_u = (\lambda n_uI + A_u)^{−1}d_u
$$

Выпишите формулы для перенастройки $q_i$ при фиксированной матрице $P$.

Ответ:

Реализуйте функцию latent_factor, которая для каждого пользователя из списка test возвращает top-k идентификаторов контента (element_uid).

Для тестирования матрицы P и Q задайте случайными 0.1 * np.random.random(...).

Исследуйте качество и время работы в зависимости от размерности $K$ пространства латентных признаков. Ведет ли увеличение $K$ к переобучению?



In [ ]:
lambda_p = 0.2
mu = 0.001
N = 20
K = 10

In [ ]:
train = transactions_train[['user_uid', 'element_uid']]
target = transactions_train['target']
test = transactions_test['user_uid'].unique()

In [ ]:
def latent_factor(train, target, test, lambda_, mu, N, K, P, Q, k = 20):
    #╰( ͡° ͜ʖ ͡° )つ──☆*:・
    pass

In [ ]:
train = pd.DataFrame({'user_uid': [1, 1, 2, 2], 'element_uid': [1, 2, 3, 4]})
target =  np.array( [0.1, 0.8, 0.2, 0.3 ])
test = np.array([1, 2])
lambda_p = 0.2
mu = 0.001
N = 20
K = 10
us = train.loc[:,'user_uid'].astype('int')
mov = train.loc[:,'element_uid'].astype('int')
U = np.unique(us)
I = np.unique(mov)
Q = 0.1 * np.ones((I.max(), K))
P = 0.1 * np.ones((U.max(), K))
assert np.all(latent_factor(train, target, test, lambda_, mu, N, K, P, Q, k = 20) == np.array([[2], [2]]))